In [1]:
import sqlite3
import pandas as pd
import os

In [2]:
conn=sqlite3.connect('data.db')

In [3]:
cursor=conn.cursor()

In [ ]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS keywords (
        keyword TEXT,
        word TEXT,
        PRIMARY KEY (keyword, word)
    )
''')


In [5]:
cursor.execute('''
    drop table if exists all_matches
''')

In [6]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS all_matches (
    filename TEXT,
    title TEXT,
    question TEXT,
    sentence TEXT,
    score REAL,
    pts REAL,
    PRIMARY KEY (filename, title, question, sentence, score, pts)
)
''')

In [7]:
cursor.execute('''CREATE TABLE IF NOT EXISTS not_matched (
               filename TEXT ,
               sentence TEXT,
               PRIMARY KEY (filename, sentence)
               )''')

In [6]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS Questionnaire (
        id INTEGER PRIMARY KEY,
        critereid TEXT NOT NULL,
        critere TEXT NOT NULL,
        sscritereid INTEGER NOT NULL,
        lestitres TEXT NOT NULL,
        questionnombre INTEGER NOT NULL,
        titrequestion TEXT NOT NULL,
        reponse TEXT NOT NULL,
        reponsedesc TEXT NOT NULL,
        maximumpts INTEGER NOT NULL
    )
    """
)


In [13]:
questionnaire = "/Users/user/Desktop/Python Scripts/questionnaire.xlsx"
data = pd.read_excel(questionnaire)
data.to_sql('Questionnaire', conn, if_exists='replace', index=False)



205

In [22]:
select_query = "SELECT * FROM Questionnaire"
df = pd.read_sql_query(select_query, conn)

df

,critereid,critere,sscritereid,lestitres,questionnombre,titrequestion,reponse,reponsedesc,maximumpts
0,A,Formation,1,Formation et expérience professionnelle préalable,1,Formation de base,1,La fonction requiert au minimum le niveau scol...,6.370
1,A,Formation,1,Formation et expérience professionnelle préalable,1,Formation de base,2,La fonction requiert une formation professionn...,8.645
2,A,Formation,1,Formation et expérience professionnelle préalable,1,Formation de base,3,La fonction requiert un certificat de fin de s...,11.375
3,A,Formation,1,Formation et expérience professionnelle préalable,1,Formation de base,4,La fonction requiert un certificat de fin de s...,15.925
4,A,Formation,1,Formation et expérience professionnelle préalable,1,Formation de base,5,La fonction requiert un Bachelor (Université o...,20.930
...,...,...,...,...,...,...,...,...,...
200,FF,Exigences de la fonction,2,Conditions d'exercice de la fonction,1,Exigences de la fonction,4,La fonction est amenée à se déplacer fréquemme...,46.800
201,FF,Fatigabilité,3,Conditions d'exercice de la fonction,1,Fatigabilité,1,La fonction est parfois soumise à des exigence...,2.340
202,FF,Fatigabilité,3,Conditions d'exercice de la fonction,1,Fatigabilité,2,La fonction est fréquemment soumise à des exig...,7.020
203,FF,Fatigabilité,3,Conditions d'exercice de la fonction,1,Fatigabilité,3,La fonction est parfois soumise à des exigence...,14.040


In [19]:
conn.commit()

In [5]:
cursor.execute('''
CREATE TABLE ResultatsJSON (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    filename TEXT UNIQUE NOT NULL,
    json_content TEXT
)
''')

conn.commit()
conn.close()

In [6]:
conn.close()

In [14]:
import json

def read_json(connection, filename: str):
    cursor = connection.cursor()
    cursor.execute('SELECT json_content FROM ResultatsJSON WHERE filename = ?', (filename,))
    row = cursor.fetchone()
    if row:
        return json.loads(row[0])  
    else:
        return None

json_result = read_json(conn, "not_matched.json")

if json_result:
    print(json.dumps(json_result, indent=2, ensure_ascii=False))  
else:
    print("No JSON found for this filename.")

with open("not_matched.json", "w", encoding="utf-8") as f:
    json.dump(json_result, f, ensure_ascii=False, indent=2)

[
  "SMQ\nVeiller à l'application et au respect du système de gestion qualité",
  "Veille informative\n8",
  "Faire preuve de la réserve et de la discrétion qu'exige sa fonction",
  "Collaborer à la santé et aux soins corporels",
  "en secteur de soins et en secteur d'animation\nTâches liées",
  "1 Accompagner le bénéficiaire de prestations aux toilettes",
  "2 Aider le bénéficiaire de prestations à s'alimenter",
  "3 Aider le bénéficiaire de prestations à s'habiller et à se déshabiller",
  "4 Participer à l'accompagnement de personnes en fin de vie",
  "Communication\nAssurer la circulation de l'information en veillant à la qualité",
  "la fiabilité et la traçabilité de cette dernière",
  "1 Effectuer des soins corporels aux bénéficiaires de prestations",
  "coucher et repositionner les bénéficiaires de prestations",
  "3 Mesurer le poids et la taille et contrôler les signes vitaux",
  "4 Soutenir les bénéficiaires de prestations dans l'application des mesures prophylactiques",
  "5 I

In [10]:
data=cursor.execute("select * from ResultatsJSON")
len(data.fetchall()[0][2])

69702

In [10]:
KEYWORDS = {
        # 1. Formation & expérience
        "Formation de base": [
            # tronc de la version XXL
            "niveau scolaire obligatoire", "niveaux scolaires obligatoires","niveau scolaire","formation post grade",
            "AFP", "AFPs", "CFC", "CFCs", "CAP", "CAPs", "BEP", "BEPs","SAS","maturité académique","certificat de fin de secondaire"
            "baccalauréat", "baccalauréats","bac,"
            "BTS", "BTSs", "DUT", "DUTs",
            "licence", "licences",
            "Bachelor", "Bachelors",
            "Master", "Masters",
            "Mastère spécialisé", "Mastères spécialisés",
            "MBA", "MBAs",
            "Doctorat", "Doctorats", "PhD", "PhDs",
            "habilitation", "habilitations", "VAE", "VAEs",

            "BAC", "bac", "bac+1", "bac+2", "bac+3", "bac+4", "bac+5",


        ],

        "Formation complémentaire": [
            "certification", "certifications", "certifié", "certifiée","certificat", "certificats",
            "certifiés", "certifiées",
            "formation continue", "formations continues",
            "complémentaire", "complémentaires","Une formation complémentaire","formation complémentaire","formations complémentaires","Deux formations complémentaires",
            "MOOC", "MOOCs",
            "micro-credential", "micro-credentials",
            "label qualité", "labels qualité",
            "examen fédéral", "examens fédéraux",
            "équivalence diplôme", "équivalences diplômes",
            "Titre RNCP", "Titres RNCP",
            "CQP", "CQPs",
            # ajouts
            "perfectionnement", "se perfectionner", "perfectionné",
            "spécialisation","spécialisation", "spécialiser", "spécialisé",
            "certifier", "certifiera",
            "actualiser", "actualisé",
            "recycler", "recyclage", "mettre à jour"
        ],

        "Nature": [
            "peu ou pas d'expérience", "peu ou pas d'expériences","de l'expérience professionnelle","même type d'expérience","même niveau de responsabilités",
            "première expérience", "premières expériences",
            "stage significatif", "stages significatifs",
            "expérience sectorielle", "expériences sectorielles",
            "expérience managériale", "expériences managériales",
            "expertise métier", "expertises métiers",
            "polyvalence sectorielle", "polyvalences sectorielles",
            "expérience internationale", "expériences internationales",
            "expérience multi-sites", "expériences multi-sites",
            # ajouts
            "pratiquer", "pratique", "pratiqué",
            "exercer", "exercé", "expérimenter", "expérimenté",
            "parcours", "background", "antécédent"
        ],

        "Durée": ["durée minimum", "durée minimale", "durée minimale requise",
            "moins d’un an", "moins d'une année",
            "0-1 an", "0 à 1 an",
            "1-3 ans", "1 à 3 ans",
            "3-5 ans", "3 à 5 ans",
            "5-8 ans", "5 à 8 ans",
            "plus de 8 ans d'expérience ",
            "8-12 ans", "8 à 12 ans",
            "12 ans et plus",
            "junior", "juniors",
            "intermédiaire", "intermédiaires",
            "confirmé", "confirmée", "confirmés", "confirmées",
            "senior", "seniors", "sénior", "séniors",
            # ajouts
            "ancienneté", "longevité", "court", "courte",
            "long", "longue", "prolongé", "prolongée",
            "permanent", "permanente"
        ],

        # 2. Responsabilités & autonomie
        "Responsabilités de planification et de réalisation des activités à court terme": [
            "élaboration", "élaborations","élaborer", "élaboré", "élaborée", "élaborés", "élaborées",
            "planification", "planifications",
            "suivi de plans à court terme", "suivis de plans à court terme","mise en oeuvre de plans à court terme","mise en oeuvre de plans","planification à long terme","planification",
            
            "planning quotidien", "plannings quotidiens",
            "ordonnancement", "ordonnancements",
            "suivi opérationnel", "suivis opérationnels",
            "gestion du temps réel", "gestions du temps réel",
            "KPI", "KPIs", "K.P.I.", "K.P.I.s",
            "optimisation processus", "optimisations processus",
            "processus optimisation", "optimisation des processus",
            # ajouts
            "exécuter", "exécution", "coordonner", "coordination",
            "mettre en œuvre", "mise en œuvre", "implémenter", "implémentation"
        ],

        "Responsabilités de planification et de réalisation des activités à long terme": [
            "plan directeur", "plans directeurs",
            "roadmap stratégique", "roadmaps stratégiques",
            "roadmap", "roadmaps",
            "réalisations futures","recherches futures","rechereches",
            "budget pluriannuel", "budgets pluriannuels",
            "veille prospective", "veilles prospectives",
            "business plan", "business plans",
            "schéma directeur", "schémas directeurs",
            "plan d’investissement", "plans d’investissement",
            # ajouts
            "anticiper", "anticipation", "projeter", "projection",
            "concevoir", "conception", "définir", "définition",
            "élaborer", "élaboration", "visionner", "vision stratégique"
        ],

        "Autonomie de décision": [
            "applique directives", "appliqué", "appliquée", "appliqués", "appliquées",
            "propose améliorations", "proposé", "proposée", "proposés", "proposées",
            "décide", "décider", "décidé", "décidée", "décidés", "décidées",
            "budget délégué", "budgets délégués",
            "signature engageante", "signatures engageantes",
            "approbation finale", "approbations finales",
            "pleine latitude", "pleines latitudes",
            "pouvoir disciplinaire", "pouvoirs disciplinaires",
            # ajouts
            "décider", "décision", "trancher", "arbitrer", "arbitrage",
            "autoriser", "autorisation", "valider", "validation",
            "indépendant", "indépendante", "autonome", "autonomes",
            "consigne", "consignes","directive", "directives","plan de travail","contrôle d'un responsable"
        ],

        "Diversité et quantité des postes à gérer": [
            "gère directement peu de postes","peu de postes","activités semblables","peu de postes","activités diverses","grand nombre de postes",
            "aucun subordonné", "aucune subordonnée", "aucuns subordonnés",
            "gestion de proximité", "gestions de proximité",
            "management intermédiaire", "managements intermédiaires",
            "management transversal", "managements transversaux",
            "direction hiérarchique", "directions hiérarchiques",
            "leadership matriciel", "leaderships matriciels",
            "animation réseau", "animations réseau",
            "pilotage multi-équipes", "pilotages multi-équipes",
            # ajouts
            "encadrer", "encadrement", "superviser", "supervision",
            "diriger", "direction", "manager", "management",
            "mobiliser", "mobilisation", "effectif", "effectifs", "staff"
        ],

        "Rôle dans la gestion des ressources humaines": [
            "ressources humaines","transfert de connaissances","formation des collaborateurs","coordonne l'exécution","coordonne","coordonné", "coordonnée", "coordonnés", "coordonnées","coordonner",
            "évaluation des fonctions","appréciation des collaborateurs"
            "coaching", "coachings", "coach", "coachs",
            "direction", "directions","recrutement", "recrutements","rémunération"
            "gestion des ressources humaines", "gestions des ressources humaines",
            "animation d’équipe", "animations d’équipe",
            "recrutement", "recrutements",
            "onboarding", "onboardings",
            "évaluation des performances", "évaluations des performances",
            "GPEC", "G.P.E.C.",
            "gestion talents", "gestion de talents", "gestions de talents",
            "négociation sociale", "négociations sociales",
            # ajouts
            "motiver", "motivation", "coacher", "coaché",
            "développer", "développement", "accompagner", "accompagnement",
            "assessor", "discipliner", "discipline"
        ],

        "Responsabilités budgétaires": [
            "suivi financier", "suivis financiers","poste budgétaire","postes budgétaires",
            "gestion budget", "gestions budgets",
            "facturation", "facturations","paiement", "paiements",
            "gestion financière", "gestions financières",
            "gestion des coûts", "gestions des coûts",
            "gestion des dépenses", "gestions des dépenses",
            "gestion des factures", "gestions des factures",
            "traitement factures", "traitements factures",
            "suivi poste budgétaire", "suivis postes budgétaires",
            "gestion centre de coûts", "gestions centres de coûts",
            "contrôle de gestion", "contrôles de gestion",
            "élaboration budget", "élaborations budget",
            "arbitrage financier", "arbitrages financiers",
            "engagement dépenses", "engagements dépenses",
            "reporting financier", "reportings financiers",
            "analyse écarts", "analyses écarts",
            "closing comptable", "closings comptables",
            # ajouts
            "budget", "budgets", "financier", "financiers","budgetaire", "budgétaires",
            "financer", "financement", "budgéter", "budgétisation",
            "payer", "paiement", "investir", "investissement",
            "calculer", "calcul", "dépenser", "dépense"
        ],

        # 3. Impact
        "Impact externe des prestations": [
            "image de marque", "images de marque",
            "orientation client", "orientations client", "orientations clients",
            "satisfaction bénéficiaire", "satisfactions bénéficiaires",
            "impact sociétal", "impacts sociétaux",
            "responsabilité sociale", "responsabilités sociales",
            "relation publique", "relations publiques",
            "influence marché", "influences marchés",
            "engagement parties prenantes", "engagements parties prenantes",
            # ajouts
            "influencer", "influence", "promouvoir", "promotion",
            "représenter", "représentation", "défendre", "défense",
            "négocier", "négociation", "visibilité", "notoriété"
        ],

        "Impact interne des prestations": [
            "prestation de service", "prestations de services","prestation fournie","bénéficiare de la prestation","bénéficiare de prestations","bénéficiares de prestations",
            "réduction coûts", "réduction des coûts", "réductions des coûts",
            "qualité service", "qualité du service", "qualités service",
            "sécurité processus", "sécurités processus",
            "productivité", "productivités",
            "efficacité opérationnelle", "efficacités opérationnelles",
            "rentabilité", "rentabilités",
            "gestion changement", "gestion des changements", "gestions changement",
            "transformation digitale", "transformations digitales",
            # ajouts
            "optimiser", "optimisation", "améliorer", "amélioration",
            "réduire", "réduction", "sécuriser", "sécurisation",
            "stabiliser", "stabilisation", "harmoniser", "harmonisation",
            "standardiser", "standardisation", "rationaliser", "rationalisation"
        ],

        # 4. Communication & langues
        "Nature des communications internes": [
            "sujet courant", "sujets courants","l'échange d'informations","l'échange d'information","échanges d'informations",
            "briefing", "briefings","interactions"
            "reporting interne", "reportings internes",
            "animation réunion", "animations réunion",
            "synthèse", "synthèses",
            "négociation interne", "négociations internes","négociation", "négociations",
            "facilitation", "facilitations",
            "communication transverse", "communications transverses",
            "gestion de conflit", "gestions de conflits",
            "culture feedback", "cultures feedback",
            # ajouts
            "informer", "information", "échanger", "échange","négocier","négociateur",
            "communiquer", "communication", "dialoguer", "dialogue",
            "transmettre", "transmission", "expliquer", "explication",
            "coordonner", "coordination"
        ],

        "Nature des communications externes": [
            "relation client", "relations clients", "relations clientèle","sujet courant","sujets courants",
            "relations fournisseurs","explication","perceptions",
            "relations institutionnelles",
            "porte-parole", "porte-paroles",
            "représentation officielle", "représentations officielles",
            "communication de crise", "communications de crise",
            "relation presse", "relations presse",
            "lobbying",
            "community management", "community managements",
            "gestion partenariats", "gestions partenariats",
            # ajouts
            "informer", "information", "expliquer", "explication",
            "présenter", "présentation", "convaincre", "conviction",
            "persuader", "persuasion", "argumenter", "argumentation",
            "promouvoir", "promotion", "négocier", "négociation"
        ],

        "Connaissances linguistiques": [
            "A1", "A2", "B1", "B2", "C1", "C2",
            "français courant", "français courants",
            "anglais professionnel", "anglais professionnels",
            "allemand technique", "allemands techniques",
            "espagnol opérationnel", "espagnols opérationnels",
            "italien conversationnel", "italiens conversationnels",
            "langue des signes", "langues des signes","langues","langue",
            "bilingue", "bilingues","deux langues","deuxième langue","troisième langue",
            "trilingue", "trilingues",
            "multilingue", "multilingues","lecture","coversation","rédaction","français","française","langue maternelle",
            # ajouts
            "parler", "parlé", "lire", "lu", "rédiger", "rédigé","écrire ","écrit",
            "traduire", "traduction", "interpréter", "interprétation",
            "fluency", "maîtrise", "maîtriser", "maîtrisé"
        ],

        # 5. Environnement & problèmes
        "Complexité de l'environnement": [
            "procédure standard", "procédures standard", "procédures standards",
            "environnement régulé", "environnements régulés",
            "incertitude forte", "incertitudes fortes",
            "contexte VUCA", "contextes VUCA",
            "multi-site", "multi-sites", "multisite", "multisites",
            "multi-projet", "multi-projets", "multiprojet", "multiprojets",
            "environnement à risque", "environnements à risque",
            "cadre international", "cadres internationaux",
            "difficultés","adaptabilité", "flexibilité",
            "complexité", "complexités",
            # ajouts
            "gérer", "gestion", "faire face", "affronter",
            "naviguer", "navigation", "complexifier", "complexification",
            "s'adapter", "adaptation", "volatil", "incertain", "ambigu", "turbulent"
        ],

        "Evolution de l'environnement": [
            "évolution régulière","adaptation","évolution rapide","évolution très rapide","processus de travail"
            "environnement à évolution régulière", "environnements à évolution régulière",
            "environnement à évolution rapide", "environnements à évolution rapide",
            "environnement à évolution très rapide", "environnements à évolution très rapide",
            "adaptation processus", "adaptations processus",
            "transformation des pratiques", "transformations des pratiques",
            # ajouts
            "évoluer", "évolution", "changer", "changement",
            "transformer", "transformation", "adapter", "adaptation",
            "progresser", "progression", "métamorphoser", "métamorphose"
        ],

        "Diversité des missions": [
            "tâches principales","même mission","2 à 3 missions différentes","issions différentes","4 missions différentes"
            "tâche répétitive", "tâches répétitives",
            "mission variée", "missions variées",
            "poly-compétence", "poly-compétences", "polycompétence", "polycompétences",
            "projet transverse", "projets transverses",
            "portefeuille diversifié", "portefeuilles diversifiés",
            "multi-business-unit", "multi-business-units", "multi BU", "multi BUs",
            # ajouts
            "diversifier", "diversification", "varier", "variation",
            "polyvalent", "polyvalente", "multitâche", "multitâches",
            "élargir", "élargissement", "alternatif", "alternative"
        ],

        "Analyse et synthèse": [
            "base d'analyse", "bases d'analyses",
            "analyse de données", "analyses de données",
            "analyse statistique", "analyses statistiques",
            "analyse de marché", "analyses de marché",
            "analyse de performance", "analyses de performance",
            "analyse de risque", "analyses de risque",
            "analyse de tendance", "analyses de tendance",
            "analyse de processus", "analyses de processus",
            "analyse de performance", "analyses de performance",
            "analyse de la concurrence", "analyses de la concurrence",
            "analyse de l'impact", "analyses de l'impact",
            "analyse de la satisfaction", "analyses de la satisfaction",
            "analyse de la qualité", "analyses de la qualité",
            "analyse de la rentabilité", "analyses de la rentabilité",
            "analyse de la productivité", "analyses de la productivité",
            "analyse de la chaîne de valeur", "analyses de la chaîne de valeur",
            "interprétation", "interprétations","interpréter","synthèse", "synthèses",
            "analyse de la supply chain", "analyses de la supply chain","sujets parfois complexes"
            "analyse", "analyses", "analyser", "analysé", "analysée", "analysés", "analysées",
            "collecte donnée", "collecte données", "collectes de données",
            "data-crunching", "data crunching", "data-crunchings",
            "diagnostic", "diagnostics",
            "analyse", "analyses", "analyser", "analysé", "analysée", "analysés", "analysées",
            "analytique", "analytiques",
            "business intelligence", "BI",
            "tableau de bord", "tableaux de bord",
            "modélisation", "modélisations", "modéliser", "modélisé", "modélisée", "modélisés", "modélisées",
            "recommandation stratégique", "recommandations stratégiques",
            # ajouts
            "évaluer", "évaluation", "interpréter", "interprétation",
            "conclure", "conclusion", "résumer", "résumé",
            "décomposer", "décomposition", "diagnostiquer", "auditer", "audit"
        ],

        "Innovation": [
            "solutions", "solution", "solutions innovantes", "solution innovante",
            "innovation", "innovations", "innovateur", "innovatrice",
            "innovation produit", "innovations produits",
            "amélioration continue", "améliorations continues",
            "lean management", "lean",
            "design thinking",
            "idéation", "idéations",
            "proof of concept", "proofs of concept",
            "prototype", "prototypes", "prototypage", "prototypages",
            "R&D", "recherche et développement",
            "gestion innovation", "gestions innovation",
            "open innovation", "open innovations",
            "veille technologique", "veilles technologiques",
            # ajouts
            "créer", "création", "inventer", "invention",
            "concevoir", "imagination", "imaginer",
            "réinventer", "réinvention", "repenser",
            "disrupter", "disruption"
        ]
    }
    

In [12]:
for keyword, words in KEYWORDS.items():
    for word in words:
        cursor.execute('INSERT OR IGNORE INTO keywords (keyword, word) VALUES (?, ?)', (keyword, word))

conn.commit()

In [ ]:
import pandas as pd
xlsxfile = "/results/questionnaire.xlsx"
data = pd.read_excel(xlsxfile)

conn.execute("""
    CREATE TABLE Question (
        critereid TEXT,
        critere TEXT,
        sscritereid INTEGER,
        lestitres TEXT,
        questionnombre INTEGER,
        titrequestion TEXT,
        reponse INTEGER,
        reponsedesc TEXT,
        maximumpts REAL
    )
""")
data.to_sql('Question', conn, if_exists='replace', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/results/questionnaire.xlsx'